In [200]:
import pandas as pd
import numpy as np
import copy


In [201]:
train=pd.read_table("plants.csv",header=None,encoding='latin-1',delimiter=",")
#train = train.iloc[0:10000,:]
train = train.iloc[0:100,:]
train.fillna(value = 0,inplace=True)
#train=pd.read_table("mushroom.data",header=None,delimiter=",")
del train[0]
columns =[]
for idex,eachrow in train.iterrows():
    for eachcol in train.columns:
        colname = eachrow[eachcol]
        if colname not in columns and str(colname) != 'nan':
            columns.append(colname)
            
df = pd.DataFrame(columns=columns)
df
for idx,eachrow in train.iterrows():
    for eachcol in train.columns:
        val = eachrow[eachcol]
        if str(val)!='na':
            df.set_value(idx,val,1)
df.fillna(value = 0,inplace=True)

df.drop([0],axis=1,inplace=True)
print (df.columns)
print (df)


C:\Users\Surbhi\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (69) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Index(['fl', 'nc', 'ct', 'dc', 'hi', 'il', 'ky', 'la', 'md', 'mi', 'ms', 'sc',
       'va', 'pr', 'vi', 'ak', 'az', 'ca', 'co', 'ga', 'id', 'in', 'ia', 'me',
       'ma', 'mn', 'mt', 'nv', 'nh', 'nm', 'ny', 'oh', 'or', 'pa', 'ri', 'tn',
       'ut', 'vt', 'wa', 'wv', 'wi', 'wy', 'ab', 'bc', 'lb', 'mb', 'nb', 'nf',
       'nt', 'ns', 'nu', 'on', 'pe', 'qc', 'sk', 'yt', 'fraspm', 'ks', 'ne',
       'nd', 'ok', 'sd', 'tx', 'al', 'ar', 'de', 'mo', 'nj'],
      dtype='object')
    fl  nc  ct  dc  hi  il  ky  la  md  mi ...  ne  nd  ok  sd  tx  al  ar  \
0    1   1   0   0   0   0   0   0   0   0 ...   0   0   0   0   0   0   0   
1    1   1   0   0   0   0   0   0   0   0 ...   0   0   0   0   0   0   0   
2    1   1   1   1   1   1   1   1   1   1 ...   0   0   0   0   0   0   0   
3    1   1   1   1   0   1   1   1   1   1 ...   0   0   0   0   0   0   0   
4    0   0   0   0   1   0   0   0   0   0 ...   0   0   0   0   0   0   0   
5    0   1   1   0   0   0   0   0   1   1 ...   0   0 

In [202]:
train = df

In [203]:
minsup = 20

In [204]:
df.head()
train = df.copy()
train.head()

,fl,nc,ct,dc,hi,il,ky,la,md,mi,...,ne,nd,ok,sd,tx,al,ar,de,mo,nj
0,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
3,1,1,1,1,0,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [205]:
def candidate_itemsets_k1(df):
    cdir = {}
    cdir[1]={}
    cols = list(df.columns)

    cols.sort()
    cdir[1]=cols
    return cdir

candidates_dir = candidate_itemsets_k1(train)
candidates_dir[1] = [x for x in candidates_dir[1]]

In [206]:
def frequent_itemsets_k1(df):
    cols = df.columns
    fdir = {}
    sup = {}
    fdir[1]={}
    freq = {}
    freq[1] = []
    
    cols = list(df.columns)
    cols.sort()
    for column in cols:
        fdir[1] = {}
    for column in cols:
        fdir[1][column] = 0
        
    for index,row in df.iterrows():
        for eachitem in cols:
            if row[eachitem] == 1:
                fdir[1][eachitem] +=1
                
    for item in fdir[1]:
        if fdir[1][item]>=minsup:
            temp = str([item])
            sup[temp] = fdir[1][item]
            freq[1].append(item)
    
    return (freq,sup)

frequent_dir, support_dir = (frequent_itemsets_k1(train))

In [207]:
def apgen1(fk):
   # frequent_itemsets[1] * frequent_itemsets[1]
    candidates = []
    flist = []
    for itemlist in frequent_dir[1]:
            flist.append(itemlist)
            for items in frequent_dir[fk]:
                if type(items)!=list:
                    if itemlist!=items and items not in flist:
                        mylist = [itemlist,items]
                        candidates.append([itemlist,items]) 
                else:
                    mylist = [itemlist]
                    for everyitem in items:
                        if everyitem!=itemlist and everyitem not in flist:
                            mylist.append(everyitem)
                        if len(mylist)==fk+1:
                            candidates.append(mylist)
    return candidates

In [208]:
def apgen2(F_k):
    candidates = []
    flag = 0
    if F_k == 1:
        flist = []
        for itemlist in frequent_dir[F_k]:
            flist.append(itemlist)
            for everyotheritem in frequent_dir[F_k]:
                if itemlist != everyotheritem and everyotheritem not in flist:
                    candidates.append([itemlist,everyotheritem])                  
    else:
        for i in range(0,len(frequent_dir[F_k])):
            my_list=[]
            flag = 0
            
            for j in range(i+1,len(frequent_dir[F_k])):                
                for k in range(0,F_k-1):
                    if frequent_dir[F_k][i][k] != frequent_dir[F_k][j][k]:
                        flag = 1
                        break
                    else:
                        my_list.append(frequent_dir[F_k][i][k])                        
                if flag == 0:
                    my_list.append(frequent_dir[F_k][i][F_k-1])
                    my_list.append(frequent_dir[F_k][j][F_k-1])
            if my_list!=[]:
                candidates.append(my_list)
            
               
    return candidates

In [209]:
k = 1
while True:
    k+=1
    mylist = []
    candidates_dir[k] = apgen2(k-1)
    for i in range(len(candidates_dir[k])):
      
        item = candidates_dir[k][i]
        count = 0
        for ix, row in train.iterrows():
            flag = 0
            for obj in item:
                if row[obj] == 0:
                    flag = 1
            if flag == 0:
                count+=1
       
        support_dir[str(item)] = count
        mylist.append(count)

    frequent_dir[k]= []
    for i in range(len(mylist)):
        if mylist[i]>=minsup:
            frequent_dir[k].append(candidates_dir[k][i])

    if len(frequent_dir[k])==0:
        break

In [210]:
candidates_dir
candidates_count = 0
for key in candidates_dir.keys():
    for items in candidates_dir[key]:
        candidates_count+=1
print("Total Candidates: ",candidates_count)

Total Candidates:  83


In [211]:
frequent_dir
freq_count = 0
for key in frequent_dir.keys():
    for items in frequent_dir[key]:
        freq_count+=1
print("Total Frequent Items:",freq_count)

Total Frequent Items: 7


In [212]:
def apriori_rule_generation_method(arr1,F_k):
    candidates = []
    flag = 0
    if F_k == 1:
        arr = []
        for everyitem in arr1:
            arr.append(everyitem)
            for everyotheritem in arr1:
                if everyitem != everyotheritem and everyotheritem not in arr:
                    candidates.append([everyitem,everyotheritem])                  
    else:
        
        for i in range(0,len(arr1)):
            my_list=[]
            flag = 0
            
            for j in range(i+1,len(arr1)):
                my_list=[]
                for k in range(0,F_k-1):
                    if arr1[i][k] != arr1[j][k]:
                        flag = 1
                        break
                    else:
                       
                        my_list.append(arr1[i][k])                        
                if flag == 0:
                    my_list.append(arr1[i][F_k-1])
                    my_list.append(arr1[j][F_k-1])
                    candidates.append(my_list)
            
               
    return candidates


In [213]:
def ap_genrules(fk,Hm,h_count):
    k = len(fk)
    m = h_count
    global count
    if k > m+1:
        Hm1 = apriori_rule_generation_method(Hm,h_count)
        for item in Hm1:
            if sorted(list(set(fk)-set(item)))==[]:
                continue
            conf = support_dir[str(fk)] / support_dir[str(sorted(list(set(fk)-set(item))))]
            lift = conf/support_dir[str(sorted(list(set(fk)-set(item))))]
           
            if conf >= minconf :
                print(list(set(fk)-set(item)),"->",item)
                count+=1
                conf_list.append((conf,lift,str((list(set(fk)-set(item)),"->",item))))
            else:
                pass            
        ap_genrules(fk,Hm1,h_count+1)

In [214]:
minconf = 0.9
minlift = 0.001
conf_list = []
count = 0
for m in range(2,k):
    for each in frequent_dir[m]:
        H1 = [everyitem for everyitem in each]
        ap_genrules(each,H1,1)
        

In [215]:
print (count)

0


In [216]:
print(sorted(conf_list, key=lambda t: t[0],reverse=True )[:5])


[]


In [217]:
arr = []
for i in range(1,k):
    for items in frequent_dir[i]:
   #     print (items)
    #    print(frequent_dir[i+1])
        items1 = set(item)
        flag = 1
        if i<k-1:
            for itemlist in frequent_dir[i+1]:
                itemlist = set(itemlist)
                if itemlist.issubset(items1):
                    flag = 0
                    break
            if flag == 1:
                if items not in arr:
                    arr.append(items)
        if i == k-1:
            arr.append(items)
            
print ("Maximal Frequent set: ")
for every in arr:
    print(every)
print("Length of Maximal Frequent set:",len(arr))

Maximal Frequent set: 
az
ca
nm
nv
or
tx
['az', 'nm']
Length of Maximal Frequent set: 7


In [218]:

arr = []
for i in range(1,k):
    for items in frequent_dir[i]:
   #     print (items)
    #    print(frequent_dir[i+1])
       # items1 = set(item)
        flag = 1
        if type(items)!=list:
            item1 = [str(items)]
        else:
            item1 = items
            
        if i<k-1:
            for itemlist in frequent_dir[i+1]:
                #itemlist = set(itemlist)
                if support_dir[str(itemlist)]==support_dir[str(item1)]:
                    flag = 0
                    break
            if flag == 1:
                if items not in arr:
                    arr.append(items)
        if i == k-1:
            arr.append(items)
            
print ("Closed Frequent set: ")
for every in arr:
    print(every)
print('Length of Closed Frequent set:',len(arr))

Closed Frequent set: 
az
ca
nm
nv
or
tx
['az', 'nm']
Length of Closed Frequent set: 7
